# Washington Double Star Catalogue Data Reduction

Imports a csv with "WDS" columns and constrains the data to physical constraint likelihood using proper motion (P.M.) and parallax (Plx). This assumes that magnitude and separation deltas have already been constrained.

In [1]:
#imports 

from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
from astropy import units as u
import pandas as pd
import re

In [2]:
#load and read csv

file_path = "all-sky-constrained.csv"
df = pd.read_csv(file_path) # create a dataframe in pandas

In [ ]:
# loop through table

for i, row in df.iterrows():
    print(row)